In [ ]:
import numpy as np
import tensorflow as tf
import tensorflow_hub as hub
import librosa
import requests
import pandas as pd
import os
from moviepy.editor import VideoFileClip

# Load YAMNet model from TensorFlow Hub
yamnet_model_handle = 'https://tfhub.dev/google/yamnet/1'
yamnet = hub.load(yamnet_model_handle)

# Fetch class names
class_map_url = 'https://raw.githubusercontent.com/tensorflow/models/master/research/audioset/yamnet/yamnet_class_map.csv'
response = requests.get(class_map_url)
class_map_csv = response.text
class_names = [line.split(',')[2].strip().replace('"', '') for line in class_map_csv.splitlines()[1:]]

def extract_audio_from_video(video_path):
    video = VideoFileClip(video_path)
    audio_path = video_path.replace('.mp4', '.wav')
    video.audio.write_audiofile(audio_path, codec='pcm_s16le')
    return audio_path

def analyze_audio(audio_path):
    # Load the audio file
    waveform, sample_rate = librosa.load(audio_path, sr=16000)
    waveform = np.array(waveform, dtype=np.float32)

    # Predict the audio event classes using YAMNet
    scores, embeddings, spectrogram = yamnet(waveform)

    # Print significant audio events detected
    threshold = 0.1
    significant_events = []
    for i, score in enumerate(np.mean(scores, axis=0)):
        if score > threshold:
            significant_events.append(f"Class: {class_names[i]}, Score: {score}")
    return significant_events

# Load the Excel file
df = pd.read_excel("FINAL_IMSyPP_EN_644_PREPROCESSED_no_restricted_videos_features_2ktoken_w_prompt_u_desc.xlsx")
#df = df[:1]
# Dictionary to store results of already analyzed audio files
analyzed_audio = {}

video_directory = "NEW_Hatebase_dataset_downloaded_videos"

for index, row in df.iterrows():
    video_id = row['video_id']
    video_path = os.path.join(video_directory, f"{video_id}.mp4")
    if os.path.exists(video_path) and video_id not in analyzed_audio:
        audio_path = extract_audio_from_video(video_path)
        analyzed_audio[video_id] = analyze_audio(audio_path)
        os.remove(audio_path)  # Clean up the extracted audio file
    df.at[index, 'audio_events'] = analyzed_audio.get(video_id, [])

# Save the updated DataFrame to a new Excel file
#df.to_excel("FINAL_IMSyPP_EN_644_PREPROCESSED_no_restricted_videos_features_2ktoken_w_prompt_u_desc_audio.xlsx", index=False)
